# DNN 推荐系统设计

### ** *整体架构* **

从推荐系统的宏观架构来看，一般来讲，可以分为两个阶段，第一阶段为召回，即候选集生成（Candidate Generation），第二阶段为排序（Ranking）。从任务目标来看，召回阶段的目标是从海量的（比如百万级别的）数据库中筛选出一个用户可能感兴趣的物品集合，作为进一步分析和选择的候选集（比如包含几百个候选物品），而排序阶段要从这个候选集中进一步精选出一个可被用户浏览的更小的集合（比如几十个物品）。虽然我们这里统称为排序，但是一般来讲在个性化推荐而非定向搜索任务中，我们不需要对最终推荐的物品进行精确的相关性排序。换一种说法，我们可以认为召回阶段要解决的是“是什么（What）”的问题，即目标用户感兴趣的是什么样的物品，而排序阶段解决的是“怎么样的问题(How)”,即用户对某个候选物品感兴趣的程度。推荐系统中的DNN模型结构一般分为三层，即信息编码和输入层，中间隐藏层，以及输出层。隐藏层做为信息交叉和深度信息抽取层，在召回和排序阶段的结构一般是一样的。根据YouTube推荐系统的实验结果，三层的 1024Relu->512Relu->256Relu 在性能和效果上可以达到最优。信息编码和输入层以及输出层，因召回和排序阶段的推荐任务和要处理的相应信息不同而不同。

### ***召回阶段***

在召回阶段，系统要根据用户的行为信息，比如浏览，搜索，评分等，人口统计学信息和其他上下文信息，来判断哪些物品可能符合用户的喜好。所以在DNN三层结构中：
- 输入层： 用户的行为信息，人口统计学信息和其他上下问信息的直接或Embedding之后的信息，要保证输入的是稠密信息，并且被归一化到[0,1]，以确保DNN模型可以训练和收敛。
- 中间隐藏层： 1024Relu->512Relu->256Relu
- 输出层： 有训练模块和服务器模块。其中训练模块用softmax来训练预测用户感兴趣的物品类别。而隐藏层的输出结果User Embedding以及隐藏层和输出层的中间参数item Embeddings作为输入到Server中，然后在Server中采用ANN或LSH算法来快速找到用户感兴趣的物品类别。

### ***排序阶段***

在排序阶段，DNN模型所要完成的任务是精确预估目标用户对候选物品的喜好程度。所以：
- 输入层：需要有更加精细的物品特征描述，以及用户与物品的关系作为输入。同样输入的信息需要Embedding 和 归一化。
- 中间隐藏层： 1024Relu->512Relu->256Relu
- 输出层：预测用户是否会选择候选物品。根据任务的不同，具体的损失和评分函数也不同。比如Youtube推荐系统中，研发人员认为用户观看视频的时长作为标签比0,1标签更能反映用户对视频的喜爱程度；所以他们采用Weighted Logistic Regreesion 作为损失函数来模拟这个输出。